# LAB11: Programowanie dynamiczne – Wyznaczanie optymalnej wielkości partii produkcyjnej

### Zadanie 1

Implementacja zagadanienia WPP, modyfikującej poprzednie zadanie plecakowe 0/1.

In [51]:
import numpy as np

def WPP(N, g, h, q, Y_max, Y_min, y0, yk):
    """
    N - liczba miesięcy
    g - koszt produkcji
    h - koszt składowania
    q -ilosc produktów w danym miesiącu
    Y_min - pojemność minimalna magazynu
    Y_max - pojemność maksymalna magazynu
    y0 - stan na początku
    yk - stan na końcu
    """
    #inicjalizacja tablic wyjściowych zawierających ilości ładunków oraz wartości funckji 
    x = np.zeros((Y_max+1, N))
    f = np.zeros((Y_max+1, N))
    #inicjalizacja macierzy wartości funckji z nieskończonościa
    f.fill(np.inf)
    #pomocnicze zmienne okreslające maksymalne i minimalne wartości zmiennej decyzyjnej
    XMAX = len(g)-1
    XMIN = 0
    #iteracja po etapach i= <0, N-1>, ale etapy wykonują się od końca (reverse_idx)
    for i in range(N): #etap
        for j in range(Y_min, Y_max+1): #waga j <Y_min, Y_max>
            #pominięcie części obliczeń pierwszego etapu
            #macierze x i f są większe ale iteracja odbywa się tylko po zakresie Nx(Ymax-Ymin+1)
            #stąd finalnie kilka początkowych wierszy macierzy może być wypełnionych zerami i inf
            if i == N-1 and j > Y_min:
                x[j, i] = np.NaN 
                f[j, i] = np.inf 
                continue
            
            #index pomocniczy do iteracji od końca
            reverse_idx = N - i - 1
        
            #dla ostatniego etapu ustawiamy wartości bo nie ma wcześniejszych x ani f oraz 
            #uzupełniamy x_min oraz x_max jedną wartością zależną od końcowych wartości
            if i == 0:
                x_min = x_max = yk + q[reverse_idx] - j
            else:
                x_min = Y_min + q[reverse_idx] - j if Y_min + q[reverse_idx] - j > XMIN else XMIN
                x_max = Y_max + q[reverse_idx] - j if Y_max + q[reverse_idx] - j < XMAX else XMAX

            #przypadki uwzględniające brak spełnienia warunków i ustawienie odpowiednich wartości dla nich
            if x_min > x_max or XMIN >  x_max or x_min < XMIN or x_max > XMAX: 
                x[j, i] = np.NaN 
                f[j, i] = np.inf 
                continue

            # iteracja po wartościach xi z zakresu <x_min, x_max>
            for xi in range(x_min, x_max+1):
                #dla etapu ostatniego koszt wyliczany jest innym wzorem nie uwzględniającym wartości funckji f poprzedniego etapu
                if i == 0:
                    cost = g[xi] + h[j + xi - q[reverse_idx]]


                #oblcizenie kosztu dla pozostałych przypadków
                else:
                    #obliczenie maksymalnego zysku dla xi
                    cost = g[xi] + h[j + xi - q[reverse_idx] - Y_min] + f[j + xi - q[reverse_idx]][i-1]
                #sprawdzenie nowy zysk jest większy od obecnego dla innej wartości xi
                if cost < f[j][i]:
                    x[j][i] = xi
                    f[j][i] = cost

    #formatowanie strategi optymalnej
    optimal_strategy = ""
    #start od wartości początkowej zadanej
    yi = y0
    #iteracja po etapach od końca
    # for i in range(N-1, -1, -1):
    #     #dodwanie do strategi i-tej ilośći produktu oraz stanu
    #     optimal_strategy += f'x{N - i - 1}={x[yi - Y_min, i] - Y_min if i == N-1 else x[yi - Y_min, i]}, y{N - i - 1}={y0 if i == N-1 else yi - Y_min},\n'
    #     yi = int(yi + x[yi - Y_min, i] - q[N - i - 1])
    # s = ' '
    # optimal_strategy += f'{s :8}y{N - i}={yi - Y_min},\n'
    optimal_strategy += f'Całkowity zminimalizowany koszt wynosi: {f[Y_min][N-1]}'
    
    return x, f, optimal_strategy

#funckja wypisująca rezultaty
def print_all(x, f, optimal_strategy):
    print(f"Macierz deyczji optymalnych\n{x}")
    print(f"Macierz wartości funckji f(yi)\n{f}")
    print(f'Strategia optymalna:\n{optimal_strategy}')

N = 6
q = np.array([3, 3, 3, 3, 3, 3])
h = np.array([0, 2, 4, 6, 8, 10])
g = np.array([0, 15, 18, 19, 20, 24])

# #Testowy zestaw z wykładu
print_all(*WPP(N, g, h, q, Y_max=4, Y_min=0, y0=0, yk=0))

Macierz deyczji optymalnych
[[ 3.  3.  4.  3.  3.  4.]
 [ 2.  5.  5.  5.  5. nan]
 [ 1.  4.  4.  4.  4. nan]
 [ 0.  0.  0.  0.  0. nan]
 [nan  0.  0.  0.  0. nan]]
Macierz wartości funckji f(yi)
[[ 19.  38.  52.  71.  90. 104.]
 [ 18.  30.  49.  68.  82.  inf]
 [ 15.  26.  45.  64.  78.  inf]
 [  0.  19.  38.  52.  71.  inf]
 [ inf  20.  32.  51.  70.  inf]]
Strategia optymalna:
Całkowity zminimalizowany koszt wynosi: 104.0


### Zadanie 2

Zadania obliczeniowe wykonane najpierw dla problemu z konspektu, następnie dla własnego 6-miesięcznego:

In [52]:
g = np.array([2, 8, 12, 15, 17, 20]) 
h = np.array([1, 2, 3, 4]) 
q = np.array([4, 2, 6, 5])
N = 4 
Y_min = 2 
Y_max = 5 
y0 = 4 
yk = 3 
print_all(*WPP(N, g, h, q, Y_max, Y_min, y0, yk))

Macierz deyczji optymalnych
[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [nan nan  4.  4.]
 [ 5.  0.  3. nan]
 [ 4.  5.  2. nan]
 [ 3.  4.  1. nan]]
Macierz wartości funckji f(yi)
[[inf inf inf inf]
 [inf inf inf inf]
 [inf inf 66. 84.]
 [24. inf 64. inf]
 [21. 46. 61. inf]
 [19. 43. 57. inf]]
Strategia optymalna:
Całkowity zminimalizowany koszt wynosi: 84.0


Zadanie obliczeniowe dla 6 miesięcy, nieliniowego kosztu produkcji g, składowania h oraz ilości szt/miesiąc q. Przy założeniu minimalnej pojemniści magazynu 1 oraz maksymalnej 6 oraz stanie początkowym, który jest równy końcowemu 3.

In [53]:
N = 6
q = np.array([3, 3, 3, 3, 3, 3])
h = np.array([0, 2, 4, 6, 8, 10])
g = np.array([0, 15, 18, 19, 20])

# #Testowy zestaw z wykładu
print_all(*WPP(N, g, h, q, Y_max=4, Y_min=0, y0=1, yk=2))

Macierz deyczji optymalnych
[[nan  4.  3.  3.  3.  4.]
 [ 4.  3.  2.  2.  4. nan]
 [ 3.  2.  1.  4.  4. nan]
 [ 2.  1.  0.  0.  0. nan]
 [ 1.  0.  0.  0.  0. nan]]
Macierz wartości funckji f(yi)
[[ inf  46.  65.  84. 103. 118.]
 [ 24.  45.  64.  83.  96.  inf]
 [ 23.  44.  61.  72.  91.  inf]
 [ 22.  41.  46.  65.  84.  inf]
 [ 19.  26.  47.  66.  85.  inf]]
Strategia optymalna:
Całkowity zminimalizowany koszt wynosi: 118.0


### Zadanie 3

1. Jakie modyfikacje zagadnienia można dodać, aby rozszerzyć i bardziej dostosować model problemu do rzeczywistych uwarunkowań produkcyjnych?

W przypadku każdego z wektorów kosztów, dostaw oraz magazynowania można sporządzić macierz, która różniłaby się w każdym miesiącu

2. Jaka jest złożoność obliczeniowa algorytmu?

Podobnie jak dla problemu plecakowego mamy zagnieżdzoną pętlę, która przechodzi po macierzy wymiaru "Yn x N" (Yn = Ymax - Ymin + 1, zakres magazynu x ilość miesięcy, faktycznie macierz może jest większa, ale przpadki z poza zakresu magazynu są pomijane w pierwszym warunku pętli). Co więcej dla każdego przypadku możemy mieć w najgorszym scenariuszu sprawdzenie każdej ilości xi z zakresu (x_min, x_max) gdzie x_max to maksymalna liczba produktów możliwa dow wyprodukowania w danym miesiącu. Zatem złożoność mogłaby wynosić maksymalnie O(x_max * Yn * N)